In [1]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.0.2
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkSession available as 'spark'.


In [2]:
import json

In [1]:
sc

''

In [4]:
t = sc.textFile('lab7/u.data')

In [5]:
t

lab7/u.data MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [6]:
t.take(3)

[u'196\t242\t3\t881250949',
 u'186\t302\t3\t891717742',
 u'22\t377\t1\t878887116']

In [5]:
t2=t.map(lambda x: x.split())

In [59]:
x=(1,2,3)

IndexError: tuple index out of range

In [161]:
t2.map(lambda x: (int(x[0]), int(x[1]), int(x[2]))).filter(lambda x: x[1]==600 ).take(100)

[(7, 600, 4), (932, 600, 2)]

In [6]:
t2=t2.map(lambda x: (int(x[0]), int(x[1]), int(x[2])))

In [7]:
t2.cache()

PythonRDD[2] at RDD at PythonRDD.scala:48

# ТОп 10 фильмов

In [29]:
top10_rates = t2.map(lambda x: (x[1],1)).reduceByKey(lambda a,b: a+b).takeOrdered(10, lambda x: -x[1])
t2.map(lambda x: (x[1],1)).reduceByKey(lambda a,b: a+b).takeOrdered(10, lambda x: -x[1])


[(50, 583),
 (258, 509),
 (100, 508),
 (181, 507),
 (294, 485),
 (286, 481),
 (288, 478),
 (1, 452),
 (300, 431),
 (121, 429)]

In [30]:
top10_rates_prepare = t2.map(lambda x: (x[1],1)).reduceByKey(lambda a,b: a+b)
top10_rates_prepare.takeOrdered(600)

[(1, 452),
 (2, 131),
 (3, 90),
 (4, 209),
 (5, 86),
 (6, 26),
 (7, 392),
 (8, 219),
 (9, 299),
 (10, 89),
 (11, 236),
 (12, 267),
 (13, 184),
 (14, 183),
 (15, 293),
 (16, 39),
 (17, 92),
 (18, 10),
 (19, 69),
 (20, 72),
 (21, 84),
 (22, 297),
 (23, 182),
 (24, 174),
 (25, 293),
 (26, 73),
 (27, 57),
 (28, 276),
 (29, 114),
 (30, 37),
 (31, 154),
 (32, 81),
 (33, 97),
 (34, 7),
 (35, 11),
 (36, 13),
 (37, 8),
 (38, 120),
 (39, 87),
 (40, 57),
 (41, 37),
 (42, 148),
 (43, 40),
 (44, 79),
 (45, 80),
 (46, 27),
 (47, 133),
 (48, 117),
 (49, 81),
 (50, 583),
 (51, 81),
 (52, 91),
 (53, 128),
 (54, 104),
 (55, 149),
 (56, 394),
 (57, 40),
 (58, 175),
 (59, 83),
 (60, 64),
 (61, 59),
 (62, 127),
 (63, 82),
 (64, 283),
 (65, 115),
 (66, 162),
 (67, 103),
 (68, 134),
 (69, 321),
 (70, 251),
 (71, 220),
 (72, 129),
 (73, 128),
 (74, 7),
 (75, 5),
 (76, 54),
 (77, 151),
 (78, 33),
 (79, 336),
 (80, 68),
 (81, 110),
 (82, 261),
 (83, 176),
 (84, 18),
 (85, 58),
 (86, 150),
 (87, 138),
 (88, 213)

In [31]:
for each in top10_rates:
    print (str(each[0]) + ',')

50,
258,
100,
181,
294,
286,
288,
1,
300,
121,


In [108]:
top10_rates = t2.map(lambda x: (x[1],1)).reduceByKey(lambda a,b: a+b)

'[258, 509]'

Готовим данные для обсечета среднего по фильмам, для этого посчитаем сумму оценок и их число для кажого фильма 

In [126]:
top10_average_prepare = t2.map(lambda x: (x[1],x[2])).aggregateByKey((0,0), \
              lambda acc,value: (acc[0]+value, acc[1]+1),\
              lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])).collect()

In [127]:
top_average = sc.parallelize(top10_average_prepare)

А теперь посчитаем среднюю оценку для каждого фильма

In [128]:
top10_average = top_average.map(lambda x: (x[0],(float(x[1][0])/float(x[1][1])))).takeOrdered(10, lambda x: -x[1])
top_average.map(lambda x: (x[0],(float(x[1][0])/float(x[1][1])))).takeOrdered(10, lambda x: -x[1])

[(814, 5.0),
 (1122, 5.0),
 (1500, 5.0),
 (1536, 5.0),
 (1189, 5.0),
 (1201, 5.0),
 (1293, 5.0),
 (1467, 5.0),
 (1599, 5.0),
 (1653, 5.0)]

In [129]:
top_average = top_average.map(lambda x: (x[0],(float(x[1][0])/float(x[1][1]))))

In [130]:
top_average.take(10)

[(2, 3.2061068702290076),
 (4, 3.550239234449761),
 (6, 3.576923076923077),
 (8, 3.9954337899543377),
 (10, 3.831460674157303),
 (12, 4.385767790262173),
 (14, 3.9672131147540983),
 (16, 3.2051282051282053),
 (18, 2.8),
 (20, 3.4166666666666665)]

Добавим названия фильмов по ID и отсортируем, покажем топ

In [137]:
top10_average=top_average.join(items).takeOrdered(10, lambda x: (-x[1][0], x[1][1]) )
top_average.join(items).takeOrdered(10, lambda x: (-x[1][0], x[1][1]) )

[(1536, (5.0, u'Aiqing wansui (1994)')),
 (1653, (5.0, u'Entertaining Angels: The Dorothy Day Story (1996)')),
 (814, (5.0, u'Great Day in Harlem, A (1994)')),
 (1201, (5.0, u'Marlene Dietrich: Shadow and Light (1996) ')),
 (1189, (5.0, u'Prefontaine (1997)')),
 (1467, (5.0, u'Saint of Fort Washington, The (1993)')),
 (1500, (5.0, u'Santa with Muscles (1996)')),
 (1599, (5.0, u"Someone Else's America (1995)")),
 (1293, (5.0, u'Star Kid (1997)')),
 (1122, (5.0, u'They Made Me a Criminal (1939)'))]

In [138]:
for each in top10_average:
    print (str(each[0]) + ',')

1536,
1653,
814,
1201,
1189,
1467,
1500,
1599,
1293,
1122,


Загрузим список фильмов

In [132]:
items = sc.textFile('lab7/u.item')

In [133]:
items.take(1)

[u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0']

In [134]:
items=items.map(lambda x: x.split('|'))

In [135]:
items=items.map(lambda x: (int(x[0]), x[1]))

In [136]:
items.take(3)

[(1, u'Toy Story (1995)'), (2, u'GoldenEye (1995)'), (3, u'Four Rooms (1995)')]

# Начнем считать top10_rating

Посчитаем глобальное среднее по всем фильмам

In [156]:
t2.take(4)

[(196, 242, 3), (186, 302, 3), (22, 377, 1), (244, 51, 2)]

вырежем id пользователей 

In [157]:
t3=t2.map(lambda x: (x[1],x[2]))

In [158]:
t3.cache()

PythonRDD[78] at RDD at PythonRDD.scala:48

подготовим данные для расчета глобального среднего - общая сумма оценок и число оценок

In [159]:
global_avg_prepare =t3.map(lambda x: x[1]).aggregate((0,0), \
              lambda acc,value: (acc[0]+value, acc[1]+1),\
              lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1]))
global_avg_prepare

(352986, 100000)

In [160]:
global_avg = float(global_avg_prepare[0])/float(global_avg_prepare[1])
global_avg

3.52986

Посчитаем количество голосовавших пользователей

In [161]:
users_count = len(t2.map(lambda x: x[0]).countByValue())
users_count

943

Сумма оценок и их количество для каждого фильма

In [162]:
top10_rating_prepare = t3.aggregateByKey((0,0), \
              lambda acc,value: (acc[0]+value, acc[1]+1),\
              lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1]))

In [163]:
top10_rating_prepare.take(10)

[(2, (420, 131)),
 (4, (742, 209)),
 (6, (93, 26)),
 (8, (875, 219)),
 (10, (341, 89)),
 (12, (1171, 267)),
 (14, (726, 183)),
 (16, (125, 39)),
 (18, (28, 10)),
 (20, (246, 72))]

In [164]:
def top10mu(summa, count, globalavg, globaluser):
    return (summa+globalavg*globaluser)/(globaluser+count)

In [165]:
top10mu(742 ,209,global_avg,users_count)

3.533557274305556

Посчитаем рейтинг с поправкой на недостающие данные, добавив глобальное среднее и число пользователей

In [166]:
top10_rating = top10_rating_prepare.map(lambda x: (x[0],(top10mu(float(x[1][0]),float(x[1][1]),global_avg,float(20)))))
top10_rating.take(10)

[(2, 3.2489880794701986),
 (4, 3.5484593886462883),
 (6, 3.556460869565217),
 (8, 3.9564736401673644),
 (10, 3.776121100917431),
 (12, 4.326122648083623),
 (14, 3.9241241379310345),
 (16, 3.3152067796610165),
 (18, 3.2865733333333336),
 (20, 3.441273913043478)]

Добавим название и отсортируем

In [167]:
top10_rating=top10_rating.join(items).takeOrdered(10, lambda x: (-x[1][0], x[1][1]) )
top10_rating

[(318, (4.407538364779874, u"Schindler's List (1993)")),
 (483, (4.38630114068441, u'Casablanca (1942)')),
 (64, (4.384809240924092, u'Shawshank Redemption, The (1994)')),
 (408, (4.345433333333333, u'Close Shave, A (1995)')),
 (50, (4.331006965174129, u'Star Wars (1977)')),
 (169, (4.330414492753624, u'Wrong Trousers, The (1993)')),
 (12, (4.326122648083623, u'Usual Suspects, The (1995)')),
 (603, (4.312651528384279, u'Rear Window (1954)')),
 (98, (4.252676097560975, u'Silence of the Lambs, The (1991)')),
 (127, (4.2484923787528865, u'Godfather, The (1972)'))]

In [65]:
for each in top10_rating:
    print (str(each[0]) + ',')

318,
483,
64,
408,
50,
169,
12,
603,
98,
127,


# Посчитаем top10_lower

Число оценок, фильма считали выше

In [170]:
top10_rates_prepare.takeOrdered(6)

[(1, 452), (2, 131), (3, 90), (4, 209), (5, 86), (6, 26)]

количество человек оценивших фильм положительно для каждого фильма

In [172]:
count_of_good = t3.filter(lambda x: x[1]>=4).aggregateByKey(0, \
              lambda x,y:(x+1),\
              lambda x,y:(x+y))
count_of_good.takeOrdered(6)

[(1, 321), (2, 51), (3, 34), (4, 122), (5, 39), (6, 15)]

Доля положительных отзывов

In [173]:
good_rating_rate = count_of_good.join(top10_rates_prepare).map(lambda x: (x[0],float(x[1][0])/float(x[1][1])))
good_rating_rate.takeOrdered(6)

[(1, 0.7101769911504425),
 (2, 0.3893129770992366),
 (3, 0.37777777777777777),
 (4, 0.583732057416268),
 (5, 0.45348837209302323),
 (6, 0.5769230769230769)]

In [174]:
z=2.576

In [175]:
top10_lower_prepare = good_rating_rate.join(top10_rates_prepare)
top10_lower_prepare.take(4)

[(4, (0.583732057416268, 209)),
 (8, (0.7077625570776256, 219)),
 (12, (0.8689138576779026, 267)),
 (16, (0.41025641025641024, 39))]

In [176]:
from math import sqrt

def confidence(n, p, z):
    if n == 0:
        return 0
    phat = p
    return ((phat + z*z/(2*n) - z * sqrt((phat*(1-phat)+z*z/(4*n))/n))/(1+z*z/n))

посчитаем нижнюю границу доверительного интервала для каждого фильма

In [177]:
top10_lower_prepare2 = top10_lower_prepare.map(lambda x: (x[0], confidence(x[1][1], x[1][0], z)))
top10_lower_prepare2.take(10)

[(4, 0.4946443455220198),
 (8, 0.6234208070688205),
 (12, 0.8066550869388946),
 (16, 0.2352871185721569),
 (20, 0.4064441565314605),
 (24, 0.42083014777433847),
 (28, 0.6507151818400247),
 (32, 0.5112460808880247),
 (36, 0.009095597498995961),
 (40, 0.1974979774602355)]

In [178]:
top10_lower = top10_lower_prepare2.join(items).takeOrdered(10, lambda x: (-x[1][0], x[1][1]) )
#top10_lower_prepare2.takeOrdered(10, lambda x: -x[1])

In [179]:
top10_lower

[(64, (0.8457526835447277, u'Shawshank Redemption, The (1994)')),
 (318, (0.8336980984637924, u"Schindler's List (1993)")),
 (98, (0.8334527970235316, u'Silence of the Lambs, The (1991)')),
 (479, (0.8332601854479507, u'Vertigo (1958)')),
 (483, (0.8262807973523965, u'Casablanca (1942)')),
 (50, (0.8182013061987115, u'Star Wars (1977)')),
 (603, (0.816144322908024, u'Rear Window (1954)')),
 (427, (0.8134728086457121, u'To Kill a Mockingbird (1962)')),
 (357, (0.808875556038692, u"One Flew Over the Cuckoo's Nest (1975)")),
 (12, (0.8066550869388946, u'Usual Suspects, The (1995)'))]

In [181]:
for each in top10_lower:
    print (str(each[0]) + ',')

64,
318,
98,
479,
483,
50,
603,
427,
357,
12,


# Посчитаем hist_film

In [19]:
t4 = t3.filter(lambda x: x[0]==22 ).map(lambda x: x[1])

In [20]:
count_22 = t4.countByValue()

In [21]:
count_22

defaultdict(<type 'int'>, {1: 5, 2: 14, 3: 46, 4: 98, 5: 134})

In [22]:
for x in xrange(1,6):
    print(str(count_22[x])+',')

5,
14,
46,
98,
134,


# Посчитаем hist_all

In [14]:
count_all = t3.map(lambda x: x[1]).countByValue()

In [15]:
for x in xrange(1,6):
    print(str(count_all[x])+',')

6110,
11370,
27145,
34174,
21201,


In [80]:
top10_rates = sorted(dict(top10_rates).items(), key=lambda x: x[1], reverse=True)


defaultdict(<type 'int'>, {1: 6110, 2: 11370, 3: 27145, 4: 34174, 5: 21201})

[50, 258, 100, 181, 294, 286, 288, 1, 300, 121]


In [53]:
sorted(dict(top10_rates).items(), key=lambda x: x[1], reverse=True)


[(50, 583),
 (258, 509),
 (100, 508),
 (181, 507),
 (294, 485),
 (286, 481),
 (288, 478),
 (1, 452),
 (300, 431),
 (121, 429)]

In [180]:
print(json.dumps({'top10_rates':[x for x,y in top10_rates], \
                  'hist_film': count_22.values(), \
                  'hist_all': count_all.values(), \
                  'top10_average': [or x,y in top10_average], \
                  'top10_rating': [x for x,y in top10_rating], \
                  'top10_lower': [x for x,y in top10_lower] }, sort_keys=True, indent=4))

{
    "hist_all": [
        6110, 
        11370, 
        27145, 
        34174, 
        21201
    ], 
    "hist_film": [
        5, 
        14, 
        46, 
        98, 
        134
    ], 
    "top10_average": [
        1536, 
        1653, 
        814, 
        1201, 
        1189, 
        1467, 
        1500, 
        1599, 
        1293, 
        1122
    ], 
    "top10_lower": [
        64, 
        318, 
        98, 
        479, 
        483, 
        50, 
        603, 
        427, 
        357, 
        12
    ], 
    "top10_rates": [
        50, 
        258, 
        100, 
        181, 
        294, 
        286, 
        288, 
        1, 
        300, 
        121
    ], 
    "top10_rating": [
        318, 
        483, 
        64, 
        408, 
        50, 
        169, 
        12, 
        603, 
        98, 
        127
    ]
}
